## Making Product Recommendations

In this notebook, you will use sales data to create product recommendations for customers. 

When creating recommendation models there are generally two approaches that vary only on the data you use to compute the "strength" of a recommendation:
- **Explicit ratings**: In this case, each user and product has a star rating attached. Think you might review restaurants in your favorite app.
- **Implicit ratings**: In this case each user and product has a rating that is derived from some behavioral metric. Typically that metric is a count, like number purchases of that product or the number of product page views. The actual rating is "implicit" in the sense that it is computed algorithmically instead of directly using the value provided.

In this notebook, you train a model that makes product recommendation based purchase history. For each user and product that appears in the history, you will sum the quantity of items purchased across each transaction. This sum will create an **explicit** rating for the user to product mapping, effectively your model is saying the more of a product a user buys across all transactions, the more relevant it is to that user.

The model then goes one step further, and enables you to calculate the recommendations for a user, who may not have bought the product before, but because her purchases are similar to another's, she might like the strongest recommendations inferred from other users like her. Think of this as the algorthim filling in the blanks for the user and a given product, it predicts what that rating should be between them.


### Using Spark ML's ALS Algorithm

Run the following cell to import the ALS class.


In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

Run the following cell to load the SaleSmall table from the SQL Pool. Make sure the SQL pool name (#SQL_POOL_NAME#) matches the name of your SQL Pool.

Remember, that in order to read from table like this, we need to use Scala to create a DataFrame around it.

Once you have the DataFrame, you can create a named view from it. Since named views are registered in the shared metastore, you can access view by name from both Scala and Python cells.

In this cell we use the Spark magic to run this cells content's using Scala, create the DataFrame and then register it as a view that we will use from later Python cells.


In [ ]:
%%spark
val df = spark.read.sqlanalytics("SQLPool01.wwi_poc.Sale") 
df.createOrReplaceTempView("sales")

Next, you need to get just the data you need to train the recommender.

A recommender really only operates on three fields: the customerid/userid, the itemid/productid and the rating.

In the following cell, we select just those columns, summing the quantity field to get a toal purchase count for any given product by that user in the history. This field, which we alias as numpurchases, is our rating.

Then we call `cache()` on the dataframe so that resultant dataset is cached in Spark memory (or disk) and does not have to be recomputed everytime we make a subequent query for it. Machine learning algorithms like ALS make several passes thru data, so caching the DataFrame provides a significant performance boost.

Run the following cell to prepare and cached the implicit ratings (IR) table.


In [ ]:
sample = spark.sql("SELECT productid, customerid, quantity FROM sales WHERE transactiondateid between 20150101 and 20150131")
sample.createOrReplaceTempView("salessample")

ir = spark.sql("SELECT productid, customerid, SUM(quantity) as numpurchases FROM salessample GROUP BY productid, customerid LIMIT 500000")
ir.cache()
display(ir)

In [ ]:
ir.count()

In training a model, we withold a subset of the data to use when evaluating the model. This is the test set. The train set is what we show to the algorithm for it to learn from.

The basic idea is if you show the model all of the data you have, it may effectively memorize all the answers- meaning it will perform very well if it has seen the question before, but probably perform poorly against new questions. This problem that is avoided is called overfitting the model to the data.

In the first line, we split our data into those train and test subsets.

In the sceond line, we instantiate the ALS algorithm, telling it which columns in our data are the user, item and rating. THe `maxIter` controls how many passes the training takes over the data, and the `regParam` controls how big of an adjustment the model makes during each pass. These are set to the common defaults so you can ignore those values for the purposes of this lab. 


In [ ]:
train, test = ir.randomSplit([0.7,0.3])
als = ALS(maxIter=5,regParam=0.01,userCol="customerid",itemCol="productid",ratingCol="numpurchases")
model = als.fit(train)

With a trained model in hand, we can now use it to make some recommendations. 

In reccommender systems it is very common to pre-compute the recommendations in batch and then simply query their reccommendations for a single customer later.

Run the following cell to batch compute the top 5 product recommendations for each customer, and see a sampling of the result.


In [ ]:
product_recommendations = model.recommendForAllUsers(5).selectExpr("customerid","explode(recommendations) as rec")
product_recommendations = product_recommendations.selectExpr("customerId", "rec.productid", "rec.rating")
product_recommendations.createOrReplaceTempView("recommendations")
display(product_recommendations)

Now that you have the batch recommendations, save the results to a table in a SQL Pool by running the following cell. With this, downstream applications can look up recommendations by issuing a traditional T-SQL query.

Don't forget to check the name of your SQL Pool used on Line 3.


In [ ]:
%%spark
val recommendations = spark.sql("SELECT * from recommendations")
recommendations.write.sqlanalytics("SQLPool01.wwi_poc.Recommendations", Constants.INTERNAL) 